# HW2: Algorithm Implementation and Basic Model Selection

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Data Loading

In [ ]:
dataset = pd.read_csv("virus_data.csv")
random_state = 17 + 75
train, test = train_test_split(dataset, test_size=0.2, random_state=random_state)

preprocessed_train = pd.read_csv("train_df_prepared.csv")
preprocessed_test = pd.read_csv("test_df_prepared.csv")

preprocessed_train = pd.concat([preprocessed_train.reset_index(drop=True), 
                    train['blood_type'].reset_index(drop=True)], axis=1)

preprocessed_test = pd.concat([preprocessed_test.reset_index(drop=True), 
                    test['blood_type'].reset_index(drop=True)], axis=1)

preprocessed_test


# Part 1: Basic model selection with k-Nearest Neighbors

In [4]:
def visualize_clf(clf, X, Y, title, 
                  xlabel, ylabel,
                  marker_size=50,
                  grid_length=300,
                  linewidths=None):
    import matplotlib.pyplot as plt
    import pandas as pd
    from matplotlib.colors import ListedColormap
    
    if isinstance(X, pd.DataFrame):
        X = X.to_numpy()
        
    # For internal use here, make sure labels are 0 and 1
    Y = np.ravel(Y).astype(int)
    labels = set(Y)
    assert len(labels) == 2, "Can only visualize two unique labels"

    if labels == set([-1,1]):
      Y = (Y + 1) // 2
      labels = set(Y)
    
    assert labels == set([0,1]), "Could not handle given labels"

    plt.figure(figsize=(8, 8))

    # Parameters
    n_classes = 2
    markers = ["D", "o"]
    palette = sns.color_palette("hls", 2)
    custom_cmap = ListedColormap(palette.as_hex())

    x_delta = np.abs(X[:, 0].max() - X[:, 0].min()) * 0.1
    y_delta = np.abs(X[:, 1].max() - X[:, 1].min()) * 0.1
    x_min, x_max = X[:, 0].min() - x_delta, X[:, 0].max() + x_delta
    y_min, y_max = X[:, 1].min() - y_delta, X[:, 1].max() + y_delta
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, grid_length),
                         np.linspace(y_min, y_max, grid_length))
    plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=2.5)

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    cs = plt.contourf(xx, yy, Z, cmap=custom_cmap, alpha=0.35)

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(alpha=0.75)

    # Plot the training points
    for i, color, marker in zip(labels, palette, markers):
        idx = np.where(Y == i)
        plt.scatter(X[idx, 0], X[idx, 1], color=color,
                    marker=marker,
                    edgecolor='white', s=marker_size,
                    linewidths=linewidths)

    plt.title(title, fontsize=20)
    plt.axis("tight")
    plt.show()
   

## (Q1) - Visualization and basic analysis

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

spread_pred_train = preprocessed_train[['PCR_01','PCR_08']]
spread_pred_label_train = preprocessed_train['spread']
spread_pred_test = preprocessed_test[['PCR_01','PCR_08']]
spread_pred_label_test = preprocessed_test['spread']

knn = KNeighborsClassifier(n_neighbors=1)

knn.fit(spread_pred_train, spread_pred_label_train)

visualize_clf(knn, spread_pred_train, spread_pred_label_train, "Decision Regions - Train accuracy is: {:.3f}".format(knn.score(spread_pred_train, spread_pred_label_train)), "PCR_01", "PCR_08")
visualize_clf(knn, spread_pred_test, spread_pred_label_test, "Decision Regions - Test accuracy is: {:.3f}".format(knn.score(spread_pred_test, spread_pred_label_test)), "PCR_01", "PCR_08")

## (Q2)

In [6]:
from sklearn.model_selection import cross_validate

k_list = list(range(1, 20, 2)) + list(range(20, 695, 65))
train_scores = []
validation_scores = []

for k in k_list:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_validate(knn, spread_pred_train, spread_pred_label_train, cv=8, scoring='accuracy', return_train_score=True)

    train_scores.append(scores['train_score'].mean())
    validation_scores.append(scores['test_score'].mean())


In [ ]:
plt.figure(figsize=(10, 6))
plt.semilogx(k_list, train_scores, label='Train')
plt.semilogx(k_list, validation_scores, label='Validation')
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.title('KNN: Accuracy vs k')
plt.legend()
plt.show()

k_max_ind = np.argmax(validation_scores)
k_max = k_list[k_max_ind]
print("Best k value is: ", k_max)
print("Mean train accuracy: ", train_scores[k_max_ind])
print("Mean validation accuracy: ", validation_scores[k_max_ind])

## (Q3)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=k_max)

knn.fit(spread_pred_train, spread_pred_label_train)

visualize_clf(knn, spread_pred_train, spread_pred_label_train, "Decision Regions - Train accuracy is: {:.3f}".format(knn.score(spread_pred_train, spread_pred_label_train)), "PCR_01", "PCR_08")
visualize_clf(knn, spread_pred_test, spread_pred_label_test, "Decision Regions - Test accuracy is: {:.3f}".format(knn.score(spread_pred_test, spread_pred_label_test)), "PCR_01", "PCR_08")

# Part 2: Decision trees

## (Q5)

In [9]:
id3_train = preprocessed_train[['PCR_04','PCR_06','SpecialProperty']]
id3_train_label = preprocessed_train[['risk']]
id3_test = preprocessed_test[['PCR_04','PCR_06','SpecialProperty']]
id3_test_label = preprocessed_test[['risk']]

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

id3 = DecisionTreeClassifier(criterion='entropy', max_depth=3)
id3 = id3.fit(id3_train, id3_train_label)
print("Train accuracy is: {:.3f}".format(id3.score(id3_train, id3_train_label)))
plt.figure(figsize=(15, 10))
plot_tree(id3, feature_names=['PCR_04', 'PCR_06', 'SpecialProperty'], class_names=['risk', 'no risk'], filled=True, fontsize=10)

# (Q6)

In [ ]:
from sklearn.model_selection import GridSearchCV


max_depth_range = range(5, 25)
min_samples_range = range(1, 20)
param_grid = {'max_depth': max_depth_range, 'min_samples_leaf': min_samples_range}

grid_search = GridSearchCV(DecisionTreeClassifier(criterion='entropy'), param_grid, cv=5, scoring='accuracy', return_train_score=True, error_score='raise')
grid_search.fit(id3_train, id3_train_label)


In [12]:
validation_acc_df = pd.DataFrame(grid_search.cv_results_['mean_test_score'].reshape(len(max_depth_range), len(min_samples_range)), index=max_depth_range, columns=min_samples_range)
train_acc_df = pd.DataFrame(grid_search.cv_results_['mean_train_score'].reshape(len(max_depth_range), len(min_samples_range)), index=max_depth_range, columns=min_samples_range)

In [ ]:
plt.figure(figsize=(10, 6))
h = sns.heatmap(train_acc_df, cmap='viridis', annot=True, fmt=".2f")
plt.title('Mean Training Accuracy')
plt.xlabel('min_samples_leaf')
plt.ylabel('max_depth')
plt.show()
plt.figure(figsize=(10, 6))
h = sns.heatmap(validation_acc_df, cmap='viridis', annot=True, fmt=".2f")
plt.title('Mean Validation Accuracy')
plt.xlabel('min_samples_leaf')
plt.ylabel('max_depth')
plt.show()


grid_search.cv_results_['mean_test_score'].shape

## (Q8)

In [ ]:
id3 = DecisionTreeClassifier(criterion='entropy', max_depth=8, min_samples_leaf=12)
id3 = id3.fit(id3_train, id3_train_label)
print("Train accuracy is: {:.3f}".format(id3.score(id3_train, id3_train_label)))
print("Test accuracy is: {:.3f}".format(id3.score(id3_test, id3_test_label)))


# Part 3: Linear SVM and the Polynomial kernel

In [ ]:
from SoftSVM import SoftSVM

def numerical_subgradient(w, b, C, X, y, delta=1e-4):
    w_ = w.copy()
    g_w = np.zeros_like(w_)
    orig_objective = SoftSVM.loss(w_, b, C, X, y)
    for i in range(g_w.shape[0]):
        w_[i] += delta
        perturbed_objective = SoftSVM.loss(w_, b, C, X, y)
        w_[i] -= delta
        g_w[i] = (perturbed_objective - orig_objective) / delta

    g_b = (SoftSVM.loss(w_, b + delta, C, X, y) - orig_objective) / delta
    return g_w, g_b



def compare_gradients(X, y, deltas, C=1, REPEATS=100, figsize=(10, 6)):
    residual_means = []

    for delta in deltas:
        residuals = []

        for _ in range(REPEATS):
            # Randomize vectors in which the gradient is computed
            w = np.random.randn(X.shape[1])
            b = np.random.randn(1)

            # Compute the two types of gradients
            analytic_grad = SoftSVM.subgradient(w, b, C, X, y)[0]
            numeric_grad = numerical_subgradient(w, b, C, X, y, delta=delta)[0]

            residual = np.linalg.norm(numeric_grad - analytic_grad)
            residuals.append(residual)

        residual_means.append(np.mean(residuals))

    plt.figure(figsize=figsize)
    plt.title('Residuals of analytical and numerical gradients', fontsize=22)
    plt.plot(deltas, residual_means, linewidth=3)      
    plt.yscale('log')
    plt.xscale('log')
    plt.xlabel('$\delta$', fontsize=18)
    plt.ylabel(r'$\left\Vert \nabla_{w}p_{C}\left(w,b\right) - u_{\delta} \left(w,b\right)\right\Vert$', fontsize=18)

    plt.grid(alpha=0.5)
    plt.show()


## (Q9)

In [ ]:
compare_gradients(np.array(spread_pred_train), np.array(spread_pred_label_train), deltas=np.logspace(-5, 0, 10))

## (Q11)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler

C=1e5
lr_vec = np.logspace(-11, -3, 5)
for lr in lr_vec:
    clf = SoftSVM(C=C, lr=lr)
    X_train_poly = PolynomialFeatures(degree=3,).fit_transform(spread_pred_train)
    X_train_poly = MinMaxScaler(feature_range=(-1,1)).fit_transform(X_train_poly)
    losses, accuracies = clf.fit_with_logs(np.array(X_train_poly), np.array(spread_pred_label_train), max_iter=5000)
    plt.figure(figsize=(13, 6))
    plt.subplot(121), plt.grid(alpha=0.5), plt.title ("Training Loss - lr: {:.2e}".format(lr))
    plt.semilogy(losses), plt.xlabel("Step"), plt.ylabel("Loss")
    plt.subplot(122), plt.grid(alpha=0.5), plt.title ("Training Accuracy - lr: {:.2e}".format(lr))
    plt.plot(accuracies), plt.xlabel("Step"), plt.ylabel("Accuracy")
    plt.tight_layout()
    plt.show()

## (Q12)

In [ ]:
from sklearn.pipeline import Pipeline

svm_clf = Pipeline([('feature_mapping', PolynomialFeatures(degree=3,)),
                    ('scaler', MinMaxScaler(feature_range=(-1,1))),
                    ('SVM', SoftSVM(C= 1e5 , lr=1e-7))])
svm_clf.fit(np.array(spread_pred_train), np.array(spread_pred_label_train))

In [ ]:
visualize_clf(svm_clf, spread_pred_train, spread_pred_label_train, "Decision Regions - Train accuracy is: {:.3f}".format(svm_clf.score(spread_pred_train, spread_pred_label_train)), "PCR_01", "PCR_08")
visualize_clf(svm_clf, spread_pred_test, spread_pred_label_test, "Decision Regions - Test accuracy is: {:.3f}".format(svm_clf.score(spread_pred_test, spread_pred_label_test)), "PCR_01", "PCR_08")

## (Q14)

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

svc_clf = make_pipeline(MinMaxScaler(feature_range=(-1,1)), SVC(gamma=1e-7))
svc_clf.fit(spread_pred_train, spread_pred_label_train)

visualize_clf(svc_clf, spread_pred_train, spread_pred_label_train, "Decision Regions - Train accuracy is: {:.3f}".format(svc_clf.score(spread_pred_train, spread_pred_label_train)), "PCR_01", "PCR_08")
visualize_clf(svc_clf, spread_pred_test, spread_pred_label_test, "Decision Regions - Test accuracy is: {:.3f}".format(svc_clf.score(spread_pred_test, spread_pred_label_test)), "PCR_01", "PCR_08")

## Q15

In [ ]:
svc_clf = make_pipeline(MinMaxScaler(feature_range=(-1,1)), SVC(gamma=200))
svc_clf.fit(spread_pred_train, spread_pred_label_train)

visualize_clf(svc_clf, spread_pred_train, spread_pred_label_train, "Decision Regions - Train accuracy is: {:.3f}".format(svc_clf.score(spread_pred_train, spread_pred_label_train)), "PCR_01", "PCR_08")
visualize_clf(svc_clf, spread_pred_test, spread_pred_label_test, "Decision Regions - Test accuracy is: {:.3f}".format(svc_clf.score(spread_pred_test, spread_pred_label_test)), "PCR_01", "PCR_08")

## Q 16

In [ ]:
svc_clf = make_pipeline(MinMaxScaler(feature_range=(-1,1)), SVC(gamma=5000))
svc_clf.fit(spread_pred_train, spread_pred_label_train)

visualize_clf(svc_clf, spread_pred_train, spread_pred_label_train, "Decision Regions - Train accuracy is: {:.3f}".format(svc_clf.score(spread_pred_train, spread_pred_label_train)), "PCR_01", "PCR_08")
visualize_clf(svc_clf, spread_pred_test, spread_pred_label_test, "Decision Regions - Test accuracy is: {:.3f}".format(svc_clf.score(spread_pred_test, spread_pred_label_test)), "PCR_01", "PCR_08")